In [13]:
from enum import Enum
from abc import abstractmethod
    
class InstructionCode(Enum):
    pass

class TurnInstructionCode(InstructionCode):
    L = 0
    R = 1
    
class MoveInstructionCode(Enum):
    F = 0
    
class DirectionInstructionCode(Enum):
    N = 0
    E = 1
    S = 2
    W = 3

class Instruction():
    
    def __init__(self, instructioncode, value):
        self.instructioncode = instructioncode
        self.value = value
        
    @abstractmethod
    def apply1(self, position, direction):
        pass
    
    @abstractmethod
    def apply2(self, shippos, shipdir, waypos):
        pass
    
class TurnInstruction(Instruction):
    
    def __init__(self, instructioncode, value):
        super(TurnInstruction, self).__init__(instructioncode, value)
        self.value = int(value / 90)
    
    def apply1(self, position, direction): 
        if self.instructioncode is TurnInstructionCode.L:
            directionvalue = (direction.value - self.value) % 4
        elif self.instructioncode is TurnInstructionCode.R:
            directionvalue = (direction.value + self.value) % 4
        return position, DirectionInstructionCode(directionvalue)
    
    def apply2(self, shippos, shipdir, waypos):
        # This part looks very ugly but i got tired of thinking about turning the coordinate system.
        if self.instructioncode is TurnInstructionCode.L:
            if self.value == 0:
                newwaypos = waypos
            elif self.value == 1:
                newwaypos = (-waypos[1], waypos[0])
            elif self.value == 2:
                newwaypos = (-waypos[0], -waypos[1])
            elif self.value == 3:
                newwaypos = (waypos[1], -waypos[0])
        elif self.instructioncode is TurnInstructionCode.R:  
            if self.value == 0:
                newwaypos = waypos
            elif self.value == 1:
                newwaypos = (waypos[1], -waypos[0])
            elif self.value == 2:
                newwaypos = (-waypos[0], -waypos[1])
            elif self.value == 3:
                newwaypos = (-waypos[1], waypos[0])
        return shippos, shipdir, newwaypos
        
            
class DirectionInstruction(Instruction):
        
    def apply1(self, position, direction):
        if self.instructioncode is DirectionInstructionCode.N:
            newposition = (position[0], position[1] + self.value)
        elif self.instructioncode is DirectionInstructionCode.E:
            newposition = (position[0] + self.value, position[1])
        elif self.instructioncode is DirectionInstructionCode.S:
            newposition = (position[0], position[1] - self.value)
        elif self.instructioncode is DirectionInstructionCode.W:
            newposition = (position[0] - self.value, position[1])
        return newposition, direction
    
    def apply2(self, shippos, shipdir, waypos):
        return shippos, shipdir, self.apply1(waypos, None)[0]
    
class MoveInstruction(Instruction):
    
    def __init__(self, value):
        super(MoveInstruction, self).__init__(MoveInstructionCode.F, value)
    
    def apply1(self, position, direction):
        instruction = DirectionInstruction(direction, self.value)
        return instruction.apply1(position, direction)
    
    def apply2(self, shippos, shipdir, waypos):
        newshippos = shippos
        for _ in range(self.value):
            if waypos[0] != 0:
                if waypos[0] > 0:
                    horizontalinstruction = DirectionInstruction(DirectionInstructionCode.E, waypos[0])
                elif waypos[0] < 0:
                    horizontalinstruction = DirectionInstruction(DirectionInstructionCode.W, -waypos[0])
                newshippos, shipdir = horizontalinstruction.apply1(newshippos, shipdir)
            if waypos[1] != 0:
                if waypos[1] > 0:
                    verticalinstruction = DirectionInstruction(DirectionInstructionCode.N, waypos[1])
                elif waypos[1] < 0:
                    verticalinstruction = DirectionInstruction(DirectionInstructionCode.S, -waypos[1])
                newshippos, shipdir = verticalinstruction.apply1(newshippos, shipdir)
        return newshippos, shipdir, waypos
            
class InstructionBuilder():
    
    def build(self, instructioncodechar, value):
        if instructioncodechar in ['N', 'S', 'E', 'W']:
            instructioncode = DirectionInstructionCode[instructioncodechar]
            return DirectionInstruction(instructioncode, value)
        elif instructioncodechar in ['L', 'R']:
            instructioncode = TurnInstructionCode[instructioncodechar]
            return TurnInstruction(instructioncode, value)
        elif instructioncodechar is 'F':
            instructioncode = MoveInstructionCode[instructioncodechar]
            return MoveInstruction(value)
        
class Ship():
    
    def __init__(self):
        self.position = (0,0)
        self.direction = DirectionInstructionCode.E
        self.waypos = (10, 1)
        
    def _apply1(self, instruction):
        self.position, self.direction = instruction.apply1(self.position, self.direction)
        
    def _apply2(self, instruction):
        self.position, self.direction, self.waypos = instruction.apply2(self.position, self.direction, self.waypos)
        
    def run1(self, instructionset):
        for instruction in instructionset:
            self._apply1(instruction)
        return self.position, self.direction
    
    def run2(self, instructionset):
        for instruction in instructionset:
            self._apply2(instruction)
        return self.position, self.direction, self.waypos         
    
def distance(pos):
    from scipy.spatial.distance import cityblock
    return cityblock(pos, (0,0))

filename = "./input.txt"
with open(filename) as f:
    content = [line.strip() for line in f.readlines()]

instructionbuilder = InstructionBuilder()
instructionset = [instructionbuilder.build(instructionstring[0], int(instructionstring[1:])) for instructionstring in content]
ship = Ship()
endposition, enddirection = ship.run1(instructionset)
print("Solution part 1: " + str(distance(endposition)))

ship = Ship()
endposition, enddirection, waypos = ship.run2(instructionset)
print("Solution part 2: " + str(distance(endposition)))


Solution part 1: 2458
Executing Instruction: (MoveInstructionCode.F, 98
(980, 98) (10, 1)
Executing Instruction: (DirectionInstructionCode.S, 4
(980, 98) (10, -3)
Executing Instruction: (DirectionInstructionCode.S, 4
(980, 98) (10, -7)
Executing Instruction: (TurnInstructionCode.L, 2
(980, 98) (-10, 7)
Executing Instruction: (DirectionInstructionCode.W, 4
(980, 98) (-14, 7)
Executing Instruction: (DirectionInstructionCode.S, 2
(980, 98) (-14, 5)
Executing Instruction: (TurnInstructionCode.R, 1
(980, 98) (5, 14)
Executing Instruction: (DirectionInstructionCode.E, 4
(980, 98) (9, 14)
Executing Instruction: (MoveInstructionCode.F, 60
(1520, 938) (9, 14)
Executing Instruction: (DirectionInstructionCode.E, 5
(1520, 938) (14, 14)
Executing Instruction: (TurnInstructionCode.R, 2
(1520, 938) (-14, -14)
Executing Instruction: (MoveInstructionCode.F, 100
(120, -462) (-14, -14)
Executing Instruction: (TurnInstructionCode.R, 2
(120, -462) (14, 14)
Executing Instruction: (MoveInstructionCode.F, 92


(44155, -10381) (77, 22)
Executing Instruction: (MoveInstructionCode.F, 22
(45849, -9897) (77, 22)
Executing Instruction: (DirectionInstructionCode.E, 2
(45849, -9897) (79, 22)
Executing Instruction: (MoveInstructionCode.F, 85
(52564, -8027) (79, 22)
Executing Instruction: (TurnInstructionCode.R, 1
(52564, -8027) (22, -79)
Executing Instruction: (DirectionInstructionCode.S, 2
(52564, -8027) (22, -81)
Executing Instruction: (TurnInstructionCode.R, 3
(52564, -8027) (81, 22)
Executing Instruction: (MoveInstructionCode.F, 16
(53860, -7675) (81, 22)
Executing Instruction: (TurnInstructionCode.R, 1
(53860, -7675) (22, -81)
Executing Instruction: (DirectionInstructionCode.W, 5
(53860, -7675) (17, -81)
Executing Instruction: (DirectionInstructionCode.S, 4
(53860, -7675) (17, -85)
Executing Instruction: (TurnInstructionCode.L, 1
(53860, -7675) (85, 17)
Executing Instruction: (DirectionInstructionCode.S, 4
(53860, -7675) (85, 13)
Executing Instruction: (DirectionInstructionCode.W, 3
(53860, -767